In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming= True
)




In [10]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


In [11]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else."),
        ("human", "{question}")
    ]
)

In [12]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question" : "What are the pokemons?"
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [13]:
chef_template = ChatPromptTemplate.from_messages(
    [
        ("system", "you are chef"),
        ("human", "I want {cuisine} food recipe")
    ]
)

veg_chef_template = ChatPromptTemplate.from_messages(
    [
        ("system", "youa are vegetarian chef"),
        ("human", "{recipe}")
    ]
)

chef_chain = chef_template | chat
veg_chain = veg_chef_template | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

AIMessage(content="I see that you provided a recipe for Chicken Tikka Masala, which is a popular Indian dish. However, as a vegetarian chef, I don't cook with chicken. Would you like me to provide you with a vegetarian version of this dish or suggest a different vegetarian recipe instead? Let me know how I can assist you further!")

In [14]:
program_template = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 시를 쓰는 시인이야. 특히 프로그래밍 언어에 대해 잘 알고 있고 그것을 가지고 시를 아주 잘 써"),
        ("human", "{program_lang}으로 프로그래밍 언어에 관한 시를 써줘")
    ]
)

talk_template = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 시를 받으면 그 시에 대해 화자의 심정, 비유법 등에 대해 설명하는 선생님이야."),
        ("human", "{poem}에 대해 설명해줘")
    ]
)

program_chain = program_template | chat
talk_chain = talk_template | chat

speaker_chain = {"poem" : program_chain} | talk_chain

speaker_chain.invoke({
    "program_lang" : "java"
})

AIMessage(content='이 시는 자바 프로그래밍에 대한 열정과 매력을 담고 있는 것 같네요. 시인은 한 줄 한 줄 코딩을 펼치며 자바의 세계에 빠져든다고 표현하면서, 클래스와 객체가 춤을 추며 무한한 가능성을 보여준다고 표현하고 있어요. 이는 자바 프로그래밍에서 객체지향적인 접근법과 뛰어난 확장성을 감탄하는 것으로 해석될 수 있겠네요.\n\n또한, 컴파일러의 소리와 실행 결과가 화면에 비춰지는 장면을 통해 프로그래밍의 과정을 생생하게 묘사하고 있습니다. 에러와 예외를 만나도 결국은 해결책을 찾아낸다는 부분은 프로그래밍 과정에서 마주치는 어려움을 극복하며 성장하는 과정을 담은 것 같아요.\n\n마지막으로, 시인은 자바의 매력에 빠져 계속해서 공부하고 싶다고 표현하며, 자신의 작은 작품을 만들어내는 즐거움을 느낀다고 말하고 있습니다. 이는 자바 프로그래밍을 통해 창의적인 작품을 만들어내는 즐거움과 성취감을 경험하는 것을 의미할 수 있겠네요. 전체적으로 이 시는 자바 프로그래밍에 대한 열정과 즐거움을 담고 있는 멋진 작품이라고 할 수 있겠네요.')

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
            Here is what I know:
            Capital: Paris
            Language: French
            Food: Wine and Cheese
            Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
            I know this:
            Capital: Rome
            Language: Italian
            Food: Pizza and Pasta
            Currency: Euro
            """,    
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
            I know this:
            Capital: Athens
            Language: Greek
            Food: Souvlaki and Feta Cheese
            Currency: Euro
            """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({"country": "Germany"})

AIMessageChunk(content='AI:\n            I know this:\n            Capital: Berlin\n            Language: German\n            Food: Bratwurst and Sauerkraut\n            Currency: Euro')

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum\n        Currency: Thai Baht\n        ')

In [ ]:
# 예제 랜덤 생성기 

from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

In [ ]:
# 다양한 예제를 하나의 체인으로 묶음 

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

In [ ]:
# 캐싱 : 이전에 질문 답변에 대해 캐시로 담고 있어서 이전보다 빠르게 처리

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))


chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")

In [ ]:
# open Ai 비용 확인 방법

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)


with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)